In [1]:
import requests
from scrapy.http import TextResponse
import pandas as pd
import time

Problem 1

In [13]:
def one_page_scraper(response):
    base_url = 'http://books.toscrape.com/catalogue/';
    pic_base_url = "http://books.toscrape.com"
    titles = response.css(".product_pod h3 a::attr(title)").extract()
    ratings = [i.replace("star-rating ","") for i in response.css(".star-rating::attr(class)").extract()]
    prices = [float(i.replace("Â£","")) for i in response.css(".product_price .price_color::text").extract()]
    book_links = [base_url+i for i in response.css(".product_pod h3 a::attr(href)").extract()]
    picture_links = [i.replace("..", pic_base_url) for i in response.css(".image_container a img::attr(src)").extract()]
    in_stock = [i.replace("\n","").strip() for i in response.css(".availability::text").extract() if i!='\n    ']
    genres = []
    descriptions = []
    for i in book_links:
        book_details = book_detail_scraper(i)
        genres.append(book_details[0])
        descriptions.append(book_details[1])
        time.sleep(0.4)
        
    return {"Title":titles,"Rating": ratings,"Price":prices, "Book Link": book_links, 
            "Picture Link": picture_links,"Availability": in_stock, "Genre": genres, "Description": descriptions};

In [14]:
def book_detail_scraper(URL):
    page = requests.get(URL)
    response = TextResponse(body=page.text,url=URL,encoding="utf-8")
    genre = response.css(".breadcrumb li:nth-child(3) a::text").extract()
    description = response.css("#product_description+ p::text").extract()
    genre = genre[0] if genre else "" 
    description = description[0] if description else "" 

    return genre, description
        

In [15]:
def books_scraper(URL):
    base_url = 'http://books.toscrape.com/catalogue/';
    data = []
    finished = False
    while(finished == False):
        page = requests.get(URL);
        response = TextResponse(body=page.text,url=URL,encoding="utf-8")
        data.append(pd.DataFrame(one_page_scraper(response)))
        time.sleep(0.4)
        if len(response.css(".next a::attr(href)")) > 0:
            URL = base_url + response.css(".next a::attr(href)").extract()[0]
        else:
            finished = True
      
    return pd.concat(data)

In [19]:
df = books_scraper("http://books.toscrape.com/catalogue/page-1.html")
df.to_csv('books.csv',index=False)
df

,Title,Rating,Price,Book Link,Picture Link,Availability,Genre,Description
0,A Light in the Attic,Three,51.77,http://books.toscrape.com/catalogue/a-light-in...,http://books.toscrape.com/media/cache/2c/da/2c...,In stock,Poetry,It's hard to imagine a world without A Light i...
1,Tipping the Velvet,One,53.74,http://books.toscrape.com/catalogue/tipping-th...,http://books.toscrape.com/media/cache/26/0c/26...,In stock,Historical Fiction,"""Erotic and absorbing...Written with starling ..."
2,Soumission,One,50.10,http://books.toscrape.com/catalogue/soumission...,http://books.toscrape.com/media/cache/3e/ef/3e...,In stock,Fiction,"Dans une France assez proche de la nÃ´tre, un ..."
3,Sharp Objects,Four,47.82,http://books.toscrape.com/catalogue/sharp-obje...,http://books.toscrape.com/media/cache/32/51/32...,In stock,Mystery,"WICKED above her hipbone, GIRL across her hear..."
4,Sapiens: A Brief History of Humankind,Five,54.23,http://books.toscrape.com/catalogue/sapiens-a-...,http://books.toscrape.com/media/cache/be/a5/be...,In stock,History,From a renowned historian comes a groundbreaki...
...,...,...,...,...,...,...,...,...
15,Alice in Wonderland (Alice's Adventures in Won...,One,55.53,http://books.toscrape.com/catalogue/alice-in-w...,http://books.toscrape.com/media/cache/96/ee/96...,In stock,Classics,
16,"Ajin: Demi-Human, Volume 1 (Ajin: Demi-Human #1)",Four,57.06,http://books.toscrape.com/catalogue/ajin-demi-...,http://books.toscrape.com/media/cache/09/7c/09...,In stock,Sequential Art,High school student Kei Nagai is struck dead i...
17,A Spy's Devotion (The Regency Spies of London #1),Five,16.97,http://books.toscrape.com/catalogue/a-spys-dev...,http://books.toscrape.com/media/cache/1b/5f/1b...,In stock,Historical Fiction,"In Englandâs Regency era, manners and elegan..."
18,1st to Die (Women's Murder Club #1),One,53.98,http://books.toscrape.com/catalogue/1st-to-die...,http://books.toscrape.com/media/cache/2b/41/2b...,In stock,Mystery,"James Patterson, bestselling author of the Ale..."


In [20]:
df.Price.mean()

#Avarage price is 35.07035

35.07034999999999

In [21]:
df.groupby('Genre').mean().sort_values(by=['Price'], inplace=False, ascending=False)

#In avarage the most expensive genre is "Suspense"

,Price
Genre,
Suspense,58.330000
Novels,54.810000
Politics,53.613333
Health,51.452500
New Adult,46.383333
Christian,42.496667
Sports and Games,41.166000
Self Help,40.620000
Travel,39.794545


In [22]:
df.groupby('Rating').mean()

#We can observe that higher priced books are in avarage rated higher, but the difference is not much

,Price
Rating,
Five,35.374490
Four,36.093296
One,34.561195
Three,34.692020
Two,34.810918


Problem 2

In [23]:
def one_page_book_scraper(response):
    base_url = 'https://staff.am';

    titles = response.css(".job-item-title .font_bold::text").extract()
    companies = response.css(".job-item-title .job_list_company_title::text").extract()
    deadlines = [i.replace("\n"," ").strip() for i in response.css(".job-list-deadline p:first-child::text").extract() if len(i)>3]
    locations = [i.replace("\n","").strip() for i in response.css(".job_location::text").extract() if len(i)>3]
    links = [base_url+i for i in response.css("#w0 > div > .web_item_card > a:first_child::attr(href)").extract()]

    return {"Title":titles, "Company": companies,"Deadline": deadlines,"Location": locations, "Link":links}
    #return {"Title":len(titles), "Company": len(companies),"Deadline": len(deadlines),"Location": len(locations), "Link":len(links)}


In [24]:
def job_scraper(URL):
    base_url = 'https://staff.am';
    data = []
    finished = False
    while(finished == False):
        page = requests.get(URL);
        response = TextResponse(body=page.text,url=URL,encoding="utf-8")
        data.append(pd.DataFrame(one_page_book_scraper(response)))
        time.sleep(1)
        if len(response.css(".next a::attr(href)").extract()) > 0:
            URL = base_url + response.css(".next a::attr(href)").extract()[0]
        else:
            finished = True

    return pd.concat(data)

In [25]:
df2 = job_scraper("https://staff.am/en/jobs")
df2.to_csv('jobs.csv', index = False)
df2

,Title,Company,Deadline,Location,Link
0,Junior Accountant,Step Consulting LLC,20 July 2020,Yerevan,https://staff.am/en/junior-accountant-21
1,Trainer,«Teghout» CJSC,08 July 2020,Alaverdi,https://staff.am/en/trainer-16
2,Digital Marketing Specialist,Network Zone,30 July 2020,Yerevan,https://staff.am/en/digital-marketing-speciali...
3,Անվտանգության աշխատակից (իգական),Atenk Ltd,15 July 2020,Yerevan,https://staff.am/en/anvtangutyan-asxatakic-iga...
4,Graphic Designer,Network Zone,30 July 2020,Yerevan,https://staff.am/en/graphic-designer-373
...,...,...,...,...,...
32,"Senior Software Engineer, Front-End",ServiceTitan,03 July 2020,Yerevan,https://staff.am/en/sr-software-engineer-front...
33,Senior Site Reliability Engineer,ServiceTitan,15 July 2020,Yerevan,https://staff.am/en/senior-site-reliability-en...
34,Lead Software Engineer,ServiceTitan,05 July 2020,Yerevan,https://staff.am/en/lead-software-engineer
35,QA Automation Engineer,ServiceTitan,05 July 2020,Yerevan,https://staff.am/en/qa-automation-engineer-91


In [26]:
df2.groupby("Company").count().sort_values(by=['Title'], inplace=False, ascending=False)

#The company with most job postings is Digitain with 32 postings

,Title,Deadline,Location,Link
Company,,,,
Digitain,32,32,32,32
SoftConstruct,29,29,29,29
PicsArt,23,23,23,23
ServiceTitan,16,16,16,16
TeamViewer Armenia,12,12,12,12
...,...,...,...,...
Devolon Armenia,1,1,1,1
Dexatel,1,1,1,1
Rocket Systems,1,1,1,1


In [27]:
print(df2.Title.str.contains('Data').value_counts())
print(df2.Title.str.contains('data').value_counts())

#There are 9 jobs containing the word "Data"

False    578
True       9
Name: Title, dtype: int64
False    587
Name: Title, dtype: int64
